# Phase III Code

Phase III will utilize code to determine the dimensions of the anicut structure, as well as the percolation pond. The Runoff data from Phase II will be used in order to determine the Max runoff within the past several years. Using the Max runoff value, the volume of the percolation pond will be determined. The height will be determined from analysis of the runoff, and the length then determined from that. The final area of the pond will then be determined. 

In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import os

In [2]:
from units import unit
meter = unit('m')

First we want to import the runoff data from Phase II. We have imported the csv file. Depending on the format of the csv, the codes below clean up that data so that the annual runoff is provided.

In [3]:
Runoff = pd.read_csv('Precipitation.csv')
Runoff.head().T

#Reading the runoff data produced in Phase II

,0,1,2,3,4
SUBDIVISION,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan
YEAR,1901,1902,1903,1904,1905
JAN,6.7,0,1.7,3.8,6.3
FEB,0,0,1.3,2.9,4.8
MAR,1.1,0,5.5,16.3,0.7
APR,0,0.5,0,0.7,1.3
MAY,6.1,4,4.2,11.4,0.3
JUN,3,49.1,2.7,14.6,4.9
JUL,79,27,154.8,39.8,30.1
AUG,59.2,71.3,87.1,45.6,0.6


In [4]:
#Adding the summation of expected runoff for each upcoming year and adding it as its own column
annual = [ row.JAN + row.FEB + row.MAR + row.APR + row.MAY + row.JUN + row.JUL + row.AUG + row.SEP + row.OCT + row.NOV + row.DEC for index, row in Runoff.iterrows() ]
Runoff['ANNUAL'] = annual
Runoff.head().T

,0,1,2,3,4
SUBDIVISION,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan
YEAR,1901,1902,1903,1904,1905
JAN,6.7,0,1.7,3.8,6.3
FEB,0,0,1.3,2.9,4.8
MAR,1.1,0,5.5,16.3,0.7
APR,0,0.5,0,0.7,1.3
MAY,6.1,4,4.2,11.4,0.3
JUN,3,49.1,2.7,14.6,4.9
JUL,79,27,154.8,39.8,30.1
AUG,59.2,71.3,87.1,45.6,0.6


The code below allows us to obtain the [predicted] maximum runoff that will occur within the next 50 years. This maximum value allows us to know how much water our percolation pond must be able to hold.

In [5]:
#Choosing the maximum capacity yield projected in the next 50 years
Runoff['ANNUAL'].max()

768.8

## *Design of Anicut Structure*
***

Now we want to begin to dimension the Anicut Structure. *An anicut is a dam built specifically to regulate water for irrigation*. We begin by reading in a file with some geographic information about the valley where the anicut will be built. We need information about the elevation of the valley, as the distance between two points of equal elevation on either side of the valley. 

In [6]:
AE = pd.read_csv('Anicut Elevation.csv')
AE

#AE = Anicut Elevation

,Elevation,Distance L,Distance R,Distance (English),Distance (Metric),Anicut Height (English),Anicut Height (Metric),Anicut Height (Rounded)
0,Ft,Ft,Ft,Ft,m,Ft,m,m
1,1228,606,742,136,41,1,0.30,1
2,1229,550,782,232,71,2,0.61,1
3,1230,495,823,328,100,3,0.91,1
4,1231,443,864,421,128,4,1.22,2
5,1232,393,903,510,155,5,1.52,2
6,1233,372,941,569,173,6,1.83,2
7,1234,353,969,616,188,7,2.13,3
8,1235,330,991,661,201,8,2.44,3
9,1236,310,1005,695,212,9,2.74,3


In [7]:
AE1 = AE.drop(['Distance L', 'Distance R', 'Distance (English)','Anicut Height (English)','Anicut Height (Rounded)'], axis=1)
AE1

,Elevation,Distance (Metric),Anicut Height (Metric)
0,Ft,m,m
1,1228,41,0.30
2,1229,71,0.61
3,1230,100,0.91
4,1231,128,1.22
5,1232,155,1.52
6,1233,173,1.83
7,1234,188,2.13
8,1235,201,2.44
9,1236,212,2.74


The table above records the change in the width of the valley at different elevation intervals. These widths will guide the length of the anicut, depending on the height chosen. **Below, choose a height for your anicut**. The height needs to be in increments of 0.5m for the anicut to be economically built. This height is dependendent on the height of the percolation pond. The **code will determine the required length** of the anicut. 

In [8]:
Height = 1
print(meter(Height))

1.00 m


In [9]:
if Height == 0.5:
    Length = 71
    print('Length of Dam = 71 m')
elif Height == 1:
    Length = 128
    print('Length of Dam = 128 m')
elif Height == 1.5:
    Length = 155
    print('Length of Dam = 155 m')
elif Height == 2:
    Length = 188
    print('Length of Dam = 188 m')
elif Height == 2.5: 
    Length = 212
    print('Length of Dam = 212 m')

Length of Dam = 128 m


Now that we have the *height* and *length* of the anicut, we can dimension the rest of the anicut. We want to find the **inclined height** and the **volume**. The proposed anicut strcture will be made of reinforced concrete. Unlike an earthen dam, concrete dams will not be subject to erosion on their super sturtcure. Therefore the slope of the upstream face does not need to account for erosion mitigation. The slope for the ustream and downstream faces are designed to have the same slope as to simplify the anicut deisgn. 

To choose the slope of the anicut, please input the *rise(vertical)* and *run(horizontal)* of the dam's face

In [10]:
import math

Vertical = 1.5
Horizontal = 1

Angle = math.atan(Vertical/Horizontal)*(180/math.pi)
print('The Angle for the inlcined face of the anicut is', Angle, 'Degrees')

The Angle for the inlcined face of the anicut is 56.309932474020215 Degrees


In [11]:
Face_Length = Height/math.sin(Angle*math.pi/180)
#Face_Length = The slope of the anicut will be the actual length along the face of the anicut
#Face_Length is in meters

print('The length of the face of the anicut is', meter(round(Face_Length,2)))

The length of the face of the anicut is 1.20 m


In [12]:
Face_Surface_Area = Face_Length*Length
print('The surface area of the face of the anicut is', round(Face_Surface_Area,2), 'meters squared')

The surface area of the face of the anicut is 153.84 meters squared


Now that we have the *height* and *length* of the anicut, we can determine the volume of the dam as well as the weight of the dam, using normal weight concrete. We assume the anicut's cross section is trapezoidal. In order to determine the volume we need to multiply the cross section of the anicut by the length. The equation for the cross section is that for a trapezoid. The volume of the anicut will multiplied by the unit weight of cocnrete to determine the weight of the anicut. 

In [14]:
Crest = 2
#The crest is the length of the top part of the anicut structure
Weight_Concrete = 2406.53 
#The weight of Normal Weight Concrete (kg/m^3)

In [26]:
Cross_Section = Height*(Crest+(Crest+((0.5547*Face_Length)*2)))/2

Volume_Anicut = Length*Cross_Section
print('The volume of the anicut is', round(Volume_Anicut,0), 'meters cubed')

Weight_Anicut = Volume_Anicut * Weight_Concrete 
print('The weight of the anicut is', round(Weight_Anicut,0), 'kg')

The volume of the anicut is 341.0 meters cubed
The weight of the anicut is 821429.0 kg


## Design of Percolation Pond

Now, knowing the dimensions of the anicut, we can design for the dimensions of the pond behind it. We begin by reading in a file with some geographic information about the valley where the pond will be located. We need information about the slope of the land directly behind the dam.

In [16]:
PondSlope = pd.read_excel('Slope Elevation.xlsx')
PondSlope.head().T

,0,1,2,3,4
Distance from Dam (Southern/Soutwestern) (ft),0.0,1.0000,2.0000,3.0000,4.0000
Elevation (ft),1227.0,1227.0256,1227.0512,1227.0768,1227.1024


The slope is assumed to be constant as concluded by Google Earth Pro. The code below delivers the assumed slope per ft behind the dam. That slope, labeled "Elevation (ft)" must be input into the following theta equation to find the angle of the slope. 

In [20]:
slopes = PondSlope.apply(lambda x: np.polyfit(PondSlope.index, x, 1)[0])
slopes
#Assume that the slope is constant (because it is very close)
#Slope = 0.05 ft/ft


Distance from Dam (Southern/Soutwestern) (ft)    1.000000
Elevation (ft)                                   0.028792
dtype: float64

In [21]:
theta = math.atan(0.028792)
theta

0.02878404796454591

The shape of the dam is assumed to be a volumetric triangular, due to the relatively low height. The assumption is that the slope on both sides of the pond ~0. The angle of the sloped ground behind the dam is used with simply geometry to compute the length maximum length behind that dam that can withold water from pouring over the dam. The second equation provides the length needed to withold the assumed annual runoff.

In [22]:
Length_of_Reservoir = Height/(math.tan(theta))
print('The length of the percolation pond is',round(Length_of_Reservoir,0), 'm')

Needed_Length = (2)*(Runoff['ANNUAL'].max())/(Height*Length)
print('The length needed for the percolation pond is', Needed_Length, 'm')

The length of the percolation pond is 35.0 m
The length needed for the percolation pon is 12.0125 m


From the code above, an if/else statement is used to decide if the suggested dimensions of the pond are acceptable given the predicted runoff. If dimenions are okay, model is complete. If dimenions will not hole the maximum water catchment, run model again with a different height.

In [19]:
if Needed_Length <= Length_of_Reservoir:
    print('Dimensions of Reservoir are OK!')
    print('Height of Reservoir = ', Height, 'm')
    print('Width of Dam = ', Length, 'm')
    print('Length of Reservoir = ', round(Length_of_Reservoir,0), 'm')
else:
    print('Current Dimensions will not hold maximum water catchment.')
    print('Height of Reservoir = ', Height, 'm')
    print('Width of Dam = ', Length, 'm')
    print('Length of Reservoir = ', round(Length_of_Reservoir,0), 'm')

Dimensions of Reservoir are OK!
Height of Reservoir =  1 m
Width of Dam =  128 m
Length of Reservoir =  20.0 m


End of Code.